In [ ]:
import os, sys, torch
from google.colab import drive

drive.mount("/content/gdrive")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

sys.path.append("/content/GHI_Forecasting")

from src.data_pipeline import (
    load_geo_and_build_static_features,
    load_temporal_artifacts,
    build_temporal_from_prep_csv,
    save_temporal_artifacts,
    get_wind_positions,
)

geo_dir = "/content/gdrive/MyDrive/CYL_geo"
raw_prep_dir = "/content/gdrive/MyDrive/CYL_GHI/prep_files"
artifacts_dir = "/content/gdrive/MyDrive/CYL_GHI/prep_files"

geo = load_geo_and_build_static_features(geo_dir)
vars_geo, station_files = geo.vars_geo, geo.station_files

loading_flag = True
if loading_flag:
    temporal = load_temporal_artifacts(artifacts_dir)
else:
    temporal = build_temporal_from_prep_csv(
        station_files=station_files,
        raw_prep_dir=raw_prep_dir,
        keep_doy_tod=True,
    )
    save_temporal_artifacts(artifacts_dir, temporal)

temporal_node_tensor = temporal.temporal_node_tensor
temporal_target_tensor = temporal.temporal_target_tensor
df_cols = temporal.df_cols
masks = temporal.masks

wind_dir_pos, wind_sp_pos = get_wind_positions(df_cols, masks)
print("wind_dir_pos:", wind_dir_pos, "wind_sp_pos:", wind_sp_pos)


In [ ]:
from src.Graph_build import build_static_adjacency

graph = build_static_adjacency(df_geo=geo.vars_geo, device=device, k=5, self_loops=False, topk_sym=False)
dist_matrix = graph["dist_matrix"]
theta_matrix = graph["theta_matrix"]
A_raw = graph["A_raw"]
A_static1 = graph["A_topk"]
A_static2 = graph["A_row_norm"]
A_static3 = graph["A_sym_norm"]


In [ ]:
from src.Graph_build import WindAdjacency

wind_adj = WindAdjacency(
    D_ij=dist_matrix,
    Theta_ij=theta_matrix,
    wind_speed_pos= wind_dir_pos,
    wind_dir_pos= wind_sp_pos,
    R=150.0,
    lambda_theta=1.0
)


In [ ]:
from src.temporal_autoencoder import pretrain_en_de_with_regularizers

# Example usage: pretrain embeddings on temporal tensor
# (Assumes temporal_node_tensor and masks are already defined above)
model, logs, best_epoch = pretrain_en_de_with_regularizers(
    train_tensor=temporal_node_tensor,
    val_tensor=None,
    in_dim=masks["mask_embed"].sum().item(),
    embed_dim=16,
    conv_hidden=128,
    window=6,
    use_attention=True,
    batch_size=8,
    lr=1e-3,
    epochs=20,
    device=device,
    early_stopping_patience=5,
    mask_embed=masks["mask_embed"],
    mask_cloud=masks["mask_cloud"],
    save_path=None,
    verbose=True,
)
